In [1]:
%autosave 60

Autosaving every 60 seconds


In [ ]:
# (!) ВАЖНО
# в этом ноутбуке проходит разбор кода из файла 
# agents_autogen.py
# некоторые детали могут быть упрощены и замоканы (mocks)
# для детального понимания советую разобраться в работе agents_autogen.py

In [ ]:
# так же важно проанализировать и понять логи реального запуска системы
# в папке good_example
# чтобы увидеть примеры реального взаимодействия агентов

# Семинар 4.3: Мультиагентные системы для разработки кода (Code Agents)

## 🎯 Цели занятия

1. Понять принципы работы мультиагентных систем в контексте разработки ПО
2. Изучить архитектуру системы с разделением ролей между агентами  
3. Освоить механизмы координации и взаимодействия агентов
4. Научиться мониторить и отслеживать работу агентов
5. Понять сложности построения надежных мультиагентных систем


## 🤖 Введение: Почему мультиагентные системы для разработки кода?

### Проблемы монолитных AI-решений для кодинга:

1. **Когнитивная перегрузка** - одна модель пытается делать все: планирование, кодинг, тестирование, документирование
2. **Отсутствие специализации** - нет фокуса на конкретные задачи (архитектура vs написание кода vs тестирование)
3. **Сложность контроля качества** - трудно обеспечить консистентность и проверки
4. **Ограниченная масштабируемость** - сложно добавлять новые возможности
5. **Отсутствие итеративности** - нет механизмов самокоррекции и улучшения

### Преимущества мультиагентного подхода:

✅ **Разделение ответственности** - каждый агент специализируется на своей области  
✅ **Повышение качества** - специализированные промпты дают лучшие результаты  
✅ **Возможность итераций** - агенты могут исправлять ошибки друг друга  
✅ **Масштабируемость** - легко добавлять новых агентов  
✅ **Мониторинг** - можно отслеживать работу каждого компонента  
✅ **Отказоустойчивость** - сбой одного агента не ломает всю систему


## 🏗️ Архитектура мультиагентной системы для разработки кода

### 7 специализированных агентов:

1. **🎭 Оркестратор (UserProxy)** - координирует работу, выполняет код и тесты
2. **🏗️ Архитектор (Planner)** - планирует разработку, определяет зависимости
3. **🔍 DataExtractor** - извлекает данные из интернета через веб-поиск
4. **💻 Программист (CodeWriter)** - пишет код согласно плану и требованиям
5. **🧪 Ревьюер (CodeReviewer)** - проводит code review и создает тесты
6. **📝 ТехПисатель (TechWriter)** - создает документацию и примеры использования  
7. **👨‍💼 TeamLead (ProblemSolver)** - анализирует проблемы и дает инструкции для исправления

### Ключевые особенности архитектуры:

- **Иерархическая структура** с TeamLead в роли супервизора
- **Специализированные промпты** для каждой роли
- **Валидация данных** через Pydantic модели
- **Итеративные циклы улучшения** с тестированием
- **Запуск кода в Docker контейнере** в изолированной среде
- **Система мониторинга** на каждом этапе

In [3]:
# Структуры данных для взаимодействия агентов (Pydantic модели)

from typing import List, Optional
from pydantic import BaseModel, Field

# Модель плана от Архитектора
class Plan(BaseModel):
    plan: List[str] = Field(..., description="Пошаговый план реализации")
    data_query: Optional[str] = Field(None, description="Запрос для веб-поиска")
    dependencies: List[str] = Field(default_factory=list, description="Python библиотеки")

# Модель данных от DataExtractor
class ExtractedData(BaseModel):
    price: Optional[float] = Field(None, description="Извлеченная цена")

# Модель кода от Программиста
class GeneratedCode(BaseModel):
    description: str = Field(..., description="Описание функциональности")
    code: str = Field(..., description="Полный код скрипта")

# Модель review от Ревьюера
class CodeReview(BaseModel):
    review_comments: List[str] = Field(..., description="Комментарии по коду")
    test_code: str = Field(..., description="Код тестов")
    improvements: List[str] = Field(default_factory=list, description="Улучшения")

# Пример валидации данных
example_plan = {
    "plan": [
        "Определить цену iPhone через веб-поиск",
        "Создать функцию расчета дней накопления", 
        "Добавить демонстрационный блок",
        "Написать тесты для функции"
    ],
    "data_query": "цена iPhone 15 Pro Max 256GB в России",
    "dependencies": ["requests", "json"]
}

plan_obj = Plan(**example_plan)
print("✅ Пример плана от Архитектора:")
print(f"   📋 Шагов в плане: {len(plan_obj.plan)}")
print(f"   🔍 Поисковый запрос: {plan_obj.data_query}")
print(f"   📦 Зависимости: {plan_obj.dependencies}")

✅ Пример плана от Архитектора:
   📋 Шагов в плане: 4
   🔍 Поисковый запрос: цена iPhone 15 Pro Max 256GB в России
   📦 Зависимости: ['requests', 'json']


## 🎯 Детальный разбор агентов и их специализации

### Принцип специализации промптов

Каждый агент имеет **жестко заданную роль** и **строго ограниченные выходные данные**. Это ключевое отличие от универсальных ассистентов.

#### Основные принципы дизайна промптов:

1. **Запрет на размышления** - `НЕ размышляйте, СРАЗУ верните JSON`
2. **Строгий формат вывода** - только определенная JSON структура
3. **Иерархия инструкций** - TeamLead имеет высший приоритет
4. **Специализированная терминология** для каждой роли
5. **Контекстная информация** от предыдущих агентов


# Примеры промптов агентов

# 🏗️ АРХИТЕКТОР - системный промпт]
```
architect_prompt = """
Вы — архитектор ПО. Анализируете запросы и создаёте планы разработки.

СТРОГИЕ ПРАВИЛА:
- НЕ размышляйте, СРАЗУ верните JSON
- ТОЛЬКО русский язык
- ТОЛЬКО указанный формат

ОБЯЗАТЕЛЬНЫЙ ФОРМАТ:
{
"plan": ["шаг 1", "шаг 2", "шаг 3"],
"data_query": "поисковый запрос на русском или null",
"dependencies": ["библиотека1", "библиотека2"]
}
"""
```

# 💻 ПРОГРАММИСТ - системный промпт (фрагмент)

```
programmer_prompt = """
Вы — Python-программист под руководством TeamLead. ПРИОРИТЕТ: выполнение инструкций TeamLead!

🎯 ИЕРАРХИЯ ИНСТРУКЦИЙ (по приоритету):
1. **ИНСТРУКЦИИ ОТ TEAMLEAD** — ВЫСШИЙ ПРИОРИТЕТ! Выполняйте ТОЧНО и НЕМЕДЛЕННО!
2. Техническое задание пользователя
3. Общие правила программирования

ОБЯЗАТЕЛЬНЫЙ ФОРМАТ:
{
"description": "Краткое описание функциональности",
"code": "def function():\\n    return result"
}

🚨 ОСОБО ВАЖНО: Если в сообщении есть "ИНСТРУКЦИЯ ОТ СУПЕРВИЗОРА" или "TEAMLEAD" - 
это КРИТИЧЕСКИ ВАЖНЫЕ инструкции! Выполняйте их ТОЧНО без отклонений!
"""
```

# 🧪 РЕВЬЮЕР - системный промпт (фрагмент)  

```
reviewer_prompt = """
Вы — эксперт по review кода и тестам под руководством TeamLead.

КРИТИЧЕСКИ ВАЖНО ДЛЯ ТЕСТОВ:
1. ЕСЛИ TEAMLEAD УКАЗАЛ КОНКРЕТНЫЕ ЗНАЧЕНИЯ - используйте ИХ, а не свои расчеты
2. ВНИМАТЕЛЬНО ИЗУЧИТЕ КОД перед написанием тестов
3. ВЫЧИСЛИТЕ ожидаемые результаты на основе РЕАЛЬНОЙ логики кода
4. ВСЕГДА добавляйте правильные импорты: from generated_script import function_name

ОБЯЗАТЕЛЬНЫЙ ФОРМАТ:
{
"review_comments": ["комментарий 1", "комментарий 2"],
"test_code": "полный код тестов с ИСПРАВЛЕНИЯМИ от TeamLead",
"improvements": ["улучшение 1", "улучшение 2"]
}
"""
```

📋 Промпты демонстрируют:
- ✅ Жесткое ограничение выходного формата
- ✅ Иерархию инструкций с приоритетом TeamLead
- ✅ Запрет на свободные размышления
- ✅ Специализированную терминологию для каждой роли

## 📊 Системы мониторинга и отслеживания агентов

### Многоуровневая система логирования

Мониторинг мультиагентной системы критически важен для:
- **Отладки взаимодействий** между агентами
- **Анализа производительности** каждого компонента  
- **Отслеживания стоимости** использования LLM
- **Обнаружения проблем** в цепочке выполнения

### Три уровня логирования:

1. **🎨 FancyLogger** - логи для анализа workflow
2. **📝 AutoGenRawLogger** - сырые логи всех взаимодействий AutoGen
3. **💰 TokenTracker** - отслеживание токенов и стоимости по агентам

### Что отслеживается:

- ✅ **Действия каждого агента** с временными метками
- ✅ **Переходы между фазами** разработки  
- ✅ **Циклы улучшения** и итерации
- ✅ **Ошибки и их исправления**
- ✅ **Использование токенов** по агентам
- ✅ **Стоимость операций** в реальном времени
- ✅ **Результаты тестирования** на каждой итерации


In [22]:
# Демонстрация систем мониторинга

# Пример работы FancyLogger
class MockFancyLogger:
    """Упрощенная демонстрация FancyLogger"""
    
    def __init__(self):
        self.agent_emojis = {
            "Архитектор": "🏗️",
            "DataExtractor": "🔍", 
            "Программист": "💻",
            "Ревьюер": "🧪",
            "TeamLead": "👨‍💼"
        }
    
    def log_agent_action(self, agent_name: str, action: str, details: str = ""):
        """Логирует действие агента с красивым форматированием"""
        emoji = self.agent_emojis.get(agent_name, "🤖")
        print(f"[12:34:56] {emoji} {agent_name:.<15} {action}")
        if details:
            print(f"          └─ {details}")
    
    def log_phase_start(self, phase_name: str, phase_number: int):
        """Логирует начало новой фазы"""
        print(f"\n🚀 ═══ ФАЗА {phase_number}: {phase_name.upper()} ═══")
    
    def log_test_results(self, iteration: int, exit_code: int, details: str):
        """Логирует результаты тестирования"""
        status = "✅ ПРОЙДЕНЫ" if exit_code == 0 else "❌ ПРОВАЛЕНЫ"
        print(f"🧪 Тесты (итерация {iteration}): {status}")

# Пример работы TokenTracker  
class MockTokenTracker:
    """Упрощенная демонстрация TokenTracker"""
    
    def __init__(self):
        self.agent_tokens = {
            "Архитектор": {"input": 1250, "output": 340, "calls": 1},
            "DataExtractor": {"input": 890, "output": 120, "calls": 3},
            "Программист": {"input": 2100, "output": 850, "calls": 2},
            "Ревьюер": {"input": 1800, "output": 920, "calls": 2},
            "TeamLead": {"input": 650, "output": 180, "calls": 1},
        }
        self.input_cost = 0.0002  # руб/токен
        self.output_cost = 0.0002
    
    def get_summary(self):
        """Возвращает сводку по использованию токенов"""
        total_input = sum(data["input"] for data in self.agent_tokens.values())
        total_output = sum(data["output"] for data in self.agent_tokens.values())
        total_cost = (total_input * self.input_cost) + (total_output * self.output_cost)
        
        return {
            "total_input": total_input,
            "total_output": total_output, 
            "total_cost": round(total_cost, 4),
            "by_agent": self.agent_tokens
        }

In [23]:
# Демонстрация работы
print("🎨 FANCY LOGGER - Красивые логи workflow:")
logger = MockFancyLogger()
logger.log_phase_start("Планирование", 1)
logger.log_agent_action("Архитектор", "План создан", "4 шага, 2 зависимости")
logger.log_agent_action("DataExtractor", "Данные извлечены", "Цена: 139990 руб")
logger.log_test_results(1, 0, "Все тесты пройдены")

print("\n\n💰 TOKEN TRACKER - Анализ стоимости:")
tracker = MockTokenTracker()
summary = tracker.get_summary()
print(f"📥 Общий input: {summary['total_input']:,} токенов")
print(f"📤 Общий output: {summary['total_output']:,} токенов") 
print(f"💵 Общая стоимость: {summary['total_cost']} руб.")
print(f"🔝 Самый затратный агент: Программист ({summary['by_agent']['Программист']['input'] + summary['by_agent']['Программист']['output']} токенов)")

🎨 FANCY LOGGER - Красивые логи workflow:

🚀 ═══ ФАЗА 1: ПЛАНИРОВАНИЕ ═══
[12:34:56] 🏗️ Архитектор..... План создан
          └─ 4 шага, 2 зависимости
[12:34:56] 🔍 DataExtractor.. Данные извлечены
          └─ Цена: 139990 руб
🧪 Тесты (итерация 1): ✅ ПРОЙДЕНЫ


💰 TOKEN TRACKER - Анализ стоимости:
📥 Общий input: 6,690 токенов
📤 Общий output: 2,410 токенов
💵 Общая стоимость: 1.82 руб.
🔝 Самый затратный агент: Программист (2950 токенов)


## ⚡ Сложности координации и взаимодействия агентов

### Основные проблемы мультиагентных систем:

#### 1. 🔄 **Синхронизация и завершение диалогов**
- **Проблема**: Как понять, когда агент закончил работу?
- **Решение**: Функция `_is_termination_msg()` анализирует содержимое сообщений
- **Сложность**: Различные паттерны завершения для разных агентов

#### 2. 🎯 **Валидация выходных данных**  
- **Проблема**: Агенты могут возвращать некорректный JSON или неполные данные
- **Решение**: Метод `_invoke_agent_and_validate()` с повторными попытками
- **Сложность**: До 3-5 попыток на каждого агента с разными стратегиями исправления

#### 3. 🔧 **Обработка ошибок и циклы исправлений**
- **Проблема**: Тесты падают, код не работает, импорты отсутствуют
- **Решение**: Супервизорская система с TeamLead агентом
- **Сложность**: Определение кто виноват - код или тесты?

#### 4. 🚫 **Предотвращение бесконечных циклов**
- **Проблема**: Агенты могут попасть в цикл одинаковых ошибок
- **Решение**: Отслеживание паттернов ошибок + принудительное супервизорское вмешательство
- **Сложность**: Баланс между настойчивостью и эффективностью

А это реальные примеры из моей работы в Cursor - агент попал в цикл
(даже такой большой стартап "не без греха")
### Пример 1
Мысли завершаются за 1 секунду после чего повторяются

<div style="text-align: center;">
  <img src="docs/cursor_loop_2.jpeg" alt="alt text" style="width:400px;">
</div>

### Пример 2
Пытается сделать одно и то же изменение несколько раз подряд
<div style="text-align: center;">
  <img src="docs/cursor_loop_1.jpeg" alt="alt text" style="width:400px;">
</div>

In [16]:
# попробуем провалидировать и обработать ошибки

import json
import re
from typing import Dict, Any

class AgentValidationDemo:
    """Демонстрация сложности валидации ответов агентов"""
    
    def __init__(self):
        self.max_retries = 3
        self.seen_patterns = set()
    
    def parse_agent_response(self, agent_name: str, response: str) -> Dict[str, Any]:
        """Имитация парсинга ответа агента с обработкой ошибок"""
        
        print(f"🔍 Парсинг ответа от {agent_name}:")
        print(f"   📝 Исходный ответ: '{response[:100]}...'")
        
        # Проблема 1: Размышления в ответе
        if "<think>" in response or "Let me think" in response:
            print("   ⚠️ ПРОБЛЕМА: Обнаружены размышления в ответе")
            # Очистка от размышлений
            cleaned = re.sub(r'<think>.*?</think>', '', response, flags=re.DOTALL)
            cleaned = re.sub(r'Let me think.*?\\n', '', cleaned)
            print(f"   🧹 После очистки: '{cleaned[:100]}...'")
            response = cleaned
        
        # Проблема 2: Поиск JSON в тексте
        json_match = re.search(r'\\{[^{}]*\\}', response)
        if not json_match:
            print("   ❌ ПРОБЛЕМА: JSON не найден в ответе")
            return {"error": "JSON не найден"}
        
        json_str = json_match.group(0)
        print(f"   ✅ JSON найден: '{json_str}'")
        
        # Проблема 3: Парсинг JSON
        try:
            parsed = json.loads(json_str)
            print("   ✅ JSON успешно распарсен")
            return parsed
        except json.JSONDecodeError as e:
            print(f"   ❌ ПРОБЛЕМА: Ошибка парсинга JSON: {e}")
            return {"error": f"JSON парсинг провалился: {e}"}
    
    def detect_error_pattern(self, error_log: str) -> str:
        """Определение типа ошибки для выбора стратегии исправления"""
        
        if "AssertionError" in error_log:
            return "assertion_error"  # Проблема в тестах
        elif "ImportError" in error_log or "NameError" in error_log:
            return "import_error"     # Проблема с импортами
        elif "SyntaxError" in error_log:
            return "syntax_error"     # Проблема в коде
        else:
            return "unknown_error"    # Неизвестная проблема
    
    def should_use_supervisor(self, error_pattern: str, consecutive_count: int) -> bool:
        """Определение необходимости супервизорского вмешательства"""
        
        # Если паттерн ошибки уже видели - используем супервизора
        if error_pattern in self.seen_patterns:
            print("🔄 Повторяющийся паттерн ошибки - запрос супервизора")
            return True
            
        # Если слишком много ошибок утверждений подряд
        if error_pattern == "assertion_error" and consecutive_count >= 3:
            print("🔁 Слишком много ошибок утверждений - запрос супервизора") 
            return True
            
        self.seen_patterns.add(error_pattern)
        return False

In [17]:
demo = AgentValidationDemo()

# Пример 1: Ответ с размышлениями
response_with_thinking = '''
<think>Нужно создать план для разработки калькулятора iPhone</think>
Let me think about this step by step...

{
"plan": ["Поиск цены iPhone", "Создание функции", "Написание тестов"],
"data_query": "iPhone 15 Pro Max цена",
"dependencies": ["requests"]
}
'''

print("=" * 60)
print("ПРИМЕР 1: Ответ агента с размышлениями")
result1 = demo.parse_agent_response("Архитектор", response_with_thinking)

# Пример 2: Анализ ошибок тестирования
print("\\n" + "=" * 60) 
print("ПРИМЕР 2: Анализ типов ошибок")

test_errors = [
    "AssertionError: assert 62 == 634",
    "ImportError: cannot import name 'calculate_days' from 'generated_script'",
    "AssertionError: assert 50 == 25" 
]

for i, error in enumerate(test_errors, 1):
    error_type = demo.detect_error_pattern(error)
    should_supervise = demo.should_use_supervisor(error_type, i)
    print(f"   {i}. Ошибка: {error[:50]}...")
    print(f"      Тип: {error_type}, Супервизор: {'ДА' if should_supervise else 'НЕТ'}")

print("\\n🎯 ВЫВОД: Координация агентов требует сложной логики обработки ошибок!")


ПРИМЕР 1: Ответ агента с размышлениями
🔍 Парсинг ответа от Архитектор:
   📝 Исходный ответ: '
<think>Нужно создать план для разработки калькулятора iPhone</think>
Let me think about this step b...'
   ⚠️ ПРОБЛЕМА: Обнаружены размышления в ответе
   🧹 После очистки: '

Let me think about this step by step...

{
"plan": ["Поиск цены iPhone", "Создание функции", "Напи...'
   ❌ ПРОБЛЕМА: JSON не найден в ответе
\n============================================================
ПРИМЕР 2: Анализ типов ошибок
   1. Ошибка: AssertionError: assert 62 == 634...
      Тип: assertion_error, Супервизор: НЕТ
   2. Ошибка: ImportError: cannot import name 'calculate_days' f...
      Тип: import_error, Супервизор: НЕТ
🔄 Повторяющийся паттерн ошибки - запрос супервизора
   3. Ошибка: AssertionError: assert 50 == 25...
      Тип: assertion_error, Супервизор: ДА
\n🎯 ВЫВОД: Координация агентов требует сложной логики обработки ошибок!


## 👨‍💼 Циклы улучшения с TeamLead

### Иерархическая система управления

Одна из ключевых инноваций системы - **TeamLead агент** в роли супервизора, который анализирует проблемы и дает **прямые инструкции** другим агентам.

### Алгоритм работы супервизора:

```
1. 🧪 Тесты провалились
      ↓
2. 👨‍💼 TeamLead анализирует:
   • Код программиста
   • Тесты ревьюера  
   • Логи ошибок
      ↓
3. 🎯 TeamLead определяет:
   • Кто виноват? (Программист vs Ревьюер)
   • Что именно неправильно?
   • Как исправить?
      ↓
4. 📋 TeamLead дает КОНКРЕТНЫЕ инструкции:
   "target_agent": "Ревьюер"
   "specific_instructions": "Исправить assert 62 == 634 на assert 634 == 634"
      ↓
5. 🔧 Целевой агент НЕМЕДЛЕННО выполняет инструкции
      ↓
6. 🔄 Цикл повторяется до успеха
```

### Ключевые особенности супервизорского управления:

✅ **Высший приоритет** - инструкции TeamLead превыше всего  
✅ **Конкретные указания** - не общие советы, а точные действия  
✅ **Анализ корневых причин** - определение истинного источника проблемы  
✅ **Прямое делегирование** - указание конкретного исполнителя  
✅ **Предотвращение споров** - агенты не могут игнорировать супервизора


#### Поведение до внедрения Тимлида

<div style="text-align: center;">
  <img src="docs/improve_loop_1.jpeg" alt="alt text" style="width:400px;">
</div>

#### Поведение после внедрения Тимлида

<div style="text-align: center;">
  <img src="docs/improve_loop_2.jpeg" alt="alt text" style="width:400px;">
</div>

In [18]:
class TeamLeadSupervisor:
    """Упрощенная демонстрация логики TeamLead супервизора"""
    
    def __init__(self):
        self.analysis_history = []
    
    def analyze_test_failure(self, code: str, test_code: str, error_log: str) -> dict:
        """Анализирует провал тестов и определяет виновника"""
        
        print("👨‍💼 TeamLead анализирует ситуацию...")
        print(f"📄 Код: {code[:100]}...")
        print(f"🧪 Тесты: {test_code[:100]}...")  
        print(f"❌ Ошибка: {error_log}")
        
        # Логика анализа ошибок
        if "AssertionError" in error_log:
            # Ищем неправильные ожидания в тестах
            if "assert" in error_log:
                # Парсим assert для определения проблемы
                import re
                assert_match = re.search(r'assert (\d+) == (\d+)', error_log)
                if assert_match:
                    actual, expected = assert_match.groups()
                    
                    # Проверяем логику кода
                    if "return" in code and "price" in code:
                        problem_analysis = f"Тест ожидает {expected}, но код возвращает {actual}. Нужно проверить правильность ожиданий в тесте."
                        target_agent = "Ревьюер"
                        instructions = f"Исправить assert {actual} == {expected} на правильное ожидаемое значение на основе логики кода. Если код правильный, измените тест."
                    else:
                        problem_analysis = f"Код возвращает {actual} вместо {expected}. Ошибка в логике кода."
                        target_agent = "Программист"  
                        instructions = f"Исправить логику кода чтобы возвращать {expected} вместо {actual}"
                        
                else:
                    problem_analysis = "Ошибка утверждения в тестах"
                    target_agent = "Ревьюер"
                    instructions = "Исправить неправильные ожидания в тестах"
                    
        elif "ImportError" in error_log or "NameError" in error_log:
            problem_analysis = "Проблема с импортами функций"
            target_agent = "Программист"
            instructions = "Исправить импорты и убедиться что все функции определены правильно"
            
        else:
            problem_analysis = "Неопределенная ошибка"
            target_agent = "Программист"
            instructions = "Исправить код согласно логам ошибок"
        
        decision = {
            "problem_analysis": problem_analysis,
            "target_agent": target_agent,
            "specific_instructions": instructions,
            "expected_outcome": "Тесты должны пройти успешно"
        }
        
        self.analysis_history.append(decision)
        return decision
    
    def format_supervisor_instruction(self, decision: dict) -> str:
        """Формирует супервизорскую инструкцию для агента"""
        
        return f"""
        🚨 КРИТИЧЕСКАЯ ИНСТРУКЦИЯ ОТ TEAMLEAD - ВЫПОЛНИТЬ НЕМЕДЛЕННО! 🚨
        
        ⚡ ЭТО ПРИОРИТЕТ №1! Забудьте все предыдущие подходы!
        
        📋 АНАЛИЗ ПРОБЛЕМЫ ОТ TEAMLEAD:
        {decision['problem_analysis']}
        
        🎯 ВАШИ ТОЧНЫЕ ИНСТРУКЦИИ ОТ TEAMLEAD:
        {decision['specific_instructions']}
        
        ✅ ОЖИДАЕМЫЙ РЕЗУЛЬТАТ:
        {decision['expected_outcome']}
        
        ⚠️ ПОМНИТЕ: TeamLead уже проанализировал проблему. Ваша задача - ТОЧНОЕ выполнение его инструкций!
        
        🚀 ДЕЙСТВУЙТЕ СЕЙЧАС! НЕ ДУМАЙТЕ - ВЫПОЛНЯЙТЕ!
        """

In [19]:
# Демонстрация работы супервизора
supervisor = TeamLeadSupervisor()

# Сценарий 1: Ошибка в тестах (неправильные ожидания)
print("="*70)
print("СЦЕНАРИЙ 1: Ошибка утверждений в тестах")

code_example = """
def calculate_days_for_iphone(monthly_salary):
    price = 139990.0
    daily_salary = monthly_salary / 22.5
    return int(price / daily_salary)
"""

test_example = """
def test_calculate_days():
    result = calculate_days_for_iphone(50000)
    assert result == 62  # НЕПРАВИЛЬНО! Должно быть 634
"""

error_log = "AssertionError: assert 634 == 62"

decision = supervisor.analyze_test_failure(code_example, test_example, error_log)

print(f"\n🎯 РЕШЕНИЕ TEAMLEAD:")
print(f"   Виновник: {decision['target_agent']}")
print(f"   Проблема: {decision['problem_analysis']}")
print(f"   Инструкция: {decision['specific_instructions'][:80]}...")

# Показываем итоговую инструкцию
instruction = supervisor.format_supervisor_instruction(decision)
print(f"\n📋 ИНСТРУКЦИЯ ДЛЯ {decision['target_agent'].upper()}:")
print(instruction[:300] + "...")

print("\n🏆 РЕЗУЛЬТАТ: Четкая иерархия и конкретные инструкции для исправления!")

СЦЕНАРИЙ 1: Ошибка утверждений в тестах
👨‍💼 TeamLead анализирует ситуацию...
📄 Код: 
def calculate_days_for_iphone(monthly_salary):
    price = 139990.0
    daily_salary = monthly_sala...
🧪 Тесты: 
def test_calculate_days():
    result = calculate_days_for_iphone(50000)
    assert result == 62  #...
❌ Ошибка: AssertionError: assert 634 == 62

🎯 РЕШЕНИЕ TEAMLEAD:
   Виновник: Ревьюер
   Проблема: Тест ожидает 62, но код возвращает 634. Нужно проверить правильность ожиданий в тесте.
   Инструкция: Исправить assert 634 == 62 на правильное ожидаемое значение на основе логики код...

📋 ИНСТРУКЦИЯ ДЛЯ РЕВЬЮЕР:

        🚨 КРИТИЧЕСКАЯ ИНСТРУКЦИЯ ОТ TEAMLEAD - ВЫПОЛНИТЬ НЕМЕДЛЕННО! 🚨

        ⚡ ЭТО ПРИОРИТЕТ №1! Забудьте все предыдущие подходы!

        📋 АНАЛИЗ ПРОБЛЕМЫ ОТ TEAMLEAD:
        Тест ожидает 62, но код возвращает 634. Нужно проверить правильность ожиданий в тесте.

        🎯 ВАШИ ТОЧНЫЕ ИНСТРУКЦ...

🏆 РЕЗУЛЬТАТ: Четкая иерархия и конкретные инструкции для исправления!


## 🎬 Практическая демонстрация работы системы

### Решаемая задача

**Задача**: Написать скрипт для расчета количества рабочих дней, необходимых для накопления на iPhone 15 Pro Max 256GB

### Пошаговое выполнение:

#### 📋 Фаза 1: Планирование (Архитектор)
```json
{
  "plan": [
    "Определить цену iPhone через веб-поиск",
    "Создать функцию расчета дней накопления", 
    "Добавить демонстрационный блок",
    "Написать тесты для функции"
  ],
  "data_query": "цена iPhone 15 Pro Max 256GB в России",
  "dependencies": ["requests", "json"]
}
```

#### 🔍 Фаза 2: Извлечение данных (DataExtractor)
```
- Выполняет веб-поиск по запросу
- Находит цену: 139990 рублей
- Возвращает: {"price": 139990.0}
```

#### 💻 Фаза 3: Написание кода (Программист)
```python
def calculate_days_for_iphone(monthly_salary):
    price = 139990.0  # Цена iPhone 15 Pro Max 256GB
    daily_salary = monthly_salary / 22.5
    return int(price / daily_salary)

if __name__ == '__main__':
    salary = 50000
    days = calculate_days_for_iphone(salary)
    print(f'Для накопления потребуется {days} рабочих дней')
```

#### 🧪 Фаза 4: Code Review и тесты (Ревьюер)
```python
import unittest
from generated_script import calculate_days_for_iphone

class TestCalculator(unittest.TestCase):
    def test_calculate_days_basic(self):
        result = calculate_days_for_iphone(50000)
        self.assertEqual(result, 62)  # ОШИБКА!
```


In [20]:
# Имитация workflow с проблемами и их решением
class WorkflowDemo:
    def __init__(self):
        self.iteration = 0
        self.issues_found = []
    
    def simulate_test_execution(self, code: str, test_code: str):
        """Имитация выполнения тестов"""
        self.iteration += 1
        print(f"\n🧪 ИТЕРАЦИЯ {self.iteration}: Выполнение тестов")
        
        # Вычисляем правильный результат
        # price = 139990, salary = 50000, daily = 50000/22.5 = 2222.22
        # days = 139990 / 2222.22 = 62.995... ≈ 62 (int)
        
        if "self.assertEqual(result, 62)" in test_code:
            print("   ❌ ТЕСТ ПРОВАЛЕН: AssertionError: assert 62 == 62")
            print("   🤔 Постойте... тест правильный, но где-то ошибка...")
            
            # Пересчитаем точно
            actual_days = int(139990 / (50000 / 22.5))  # = 62
            if actual_days == 62:
                print(f"   ✅ Пересчет: {actual_days} дней - тест на самом деле ПРАВИЛЬНЫЙ!")
                return 0, "Тесты пройдены"
            else:
                return 1, f"AssertionError: assert {actual_days} == 62"
        
        # Если в тесте неправильное ожидание
        elif "self.assertEqual(result, 634)" in test_code:
            print("   ❌ ТЕСТ ПРОВАЛЕН: AssertionError: assert 62 == 634")
            self.issues_found.append("incorrect_test_expectation")
            return 1, "AssertionError: assert 62 == 634"
        else:
            print("   ✅ ТЕСТЫ ПРОЙДЕНЫ!")
            return 0, "All tests passed"
    
    def demonstrate_teamlead_intervention(self, error_log: str):
        """Демонстрация вмешательства TeamLead"""
        print(f"\n👨‍💼 TEAMLEAD ВМЕШАТЕЛЬСТВО (после итерации {self.iteration}):")
        print("   📊 Анализирую проблему...")
        
        if "assert 62 == 634" in error_log:
            print("   🎯 АНАЛИЗ: Код возвращает 62, тест ожидает 634")
            print("   🧮 РАСЧЕТ: 139990 / (50000/22.5) = 139990 / 2222.22 = 62.995 ≈ 62")
            print("   💡 ВЫВОД: Код ПРАВИЛЬНЫЙ, тест НЕПРАВИЛЬНЫЙ!")
            print("   📋 ИНСТРУКЦИЯ РЕВЬЮЕРУ: Исправить assert result == 634 на assert result == 62")
            
            return {
                "target_agent": "Ревьюер",
                "instruction": "Исправить ожидаемое значение с 634 на 62"
            }
        
        return None

In [21]:
# Запуск демонстрации
demo = WorkflowDemo()

# Сначала тест с правильным ожиданием (но может показаться что неправильный)
correct_test = '''
def test_calculate_days_basic(self):
    result = calculate_days_for_iphone(50000)
    self.assertEqual(result, 62)  # На самом деле правильно!
'''

code = '''
def calculate_days_for_iphone(monthly_salary):
    price = 139990.0
    daily_salary = monthly_salary / 22.5  
    return int(price / daily_salary)
'''

exit_code, logs = demo.simulate_test_execution(code, correct_test)

if exit_code == 0:
    print("\n🎉 УСПЕХ: Система работает корректно!")
else:
    # Демонстрация неправильного теста
    print("\n🔄 ДЕМОНСТРАЦИЯ: Что если тест изначально неправильный?")
    
    wrong_test = '''
def test_calculate_days_basic(self):
    result = calculate_days_for_iphone(50000)
    self.assertEqual(result, 634)  # НЕПРАВИЛЬНО - ошибка на порядок!
'''
    
    exit_code, error_logs = demo.simulate_test_execution(code, wrong_test)
    
    if exit_code != 0:
        decision = demo.demonstrate_teamlead_intervention(error_logs)
        if decision:
            print(f"\n🔧 ИСПРАВЛЕНИЕ: {decision['target_agent']} получает инструкцию")
            print(f"   📝 Действие: {decision['instruction']}")
            
            # Имитируем исправление
            fixed_test = '''
def test_calculate_days_basic(self):
    result = calculate_days_for_iphone(50000)
    self.assertEqual(result, 62)  # ИСПРАВЛЕНО!
'''
            
            print("\n🔄 ПОВТОРНОЕ ТЕСТИРОВАНИЕ после исправления:")
            final_exit_code, final_logs = demo.simulate_test_execution(code, fixed_test)
            
            if final_exit_code == 0:
                print("\n🏆 ФИНАЛЬНЫЙ УСПЕХ: TeamLead успешно координировал исправление!")

print(f"\n📊 ИТОГО: {demo.iteration} итераций, {len(demo.issues_found)} проблем найдено и решено")


🧪 ИТЕРАЦИЯ 1: Выполнение тестов
   ❌ ТЕСТ ПРОВАЛЕН: AssertionError: assert 62 == 62
   🤔 Постойте... тест правильный, но где-то ошибка...
   ✅ Пересчет: 62 дней - тест на самом деле ПРАВИЛЬНЫЙ!

🎉 УСПЕХ: Система работает корректно!

📊 ИТОГО: 1 итераций, 0 проблем найдено и решено


## 📈 Анализ метрик и производительности системы

### Ключевые показатели мультиагентной системы:

#### 💰 Экономические метрики:
- **Стоимость токенов по агентам** - кто потребляет больше всего ресурсов
- **Общая стоимость проекта** - бюджет на разработку одного скрипта
- **ROI эффективность** - соотношение качества результата к затратам

#### ⏱️ Производительность:
- **Время выполнения** каждой фазы
- **Количество итераций** до успешного решения
- **Частота вмешательства** TeamLead супервизора
- **Успешность с первой попытки** по агентам

#### 🔄 Качество координации:
- **Процент валидных JSON ответов** от каждого агента
- **Количество повторных попыток** на агента
- **Эффективность исправлений** после инструкций TeamLead

### Типичные метрики проекта:


In [15]:
# Анализ метрик производительности мультиагентной системы

import json
from datetime import datetime, timedelta

class MetricsAnalyzer:
    """Анализатор метрик производительности агентов"""
    
    def __init__(self):
        # Имитация реальных данных из успешного проекта
        self.session_data = {
            "start_time": datetime.now() - timedelta(minutes=12),
            "end_time": datetime.now(),
            "agents_performance": {
                "Архитектор": {
                    "tokens_input": 1420, "tokens_output": 380, "calls": 1,
                    "success_rate": 100, "avg_response_time": 8.5, "retries": 0
                },
                "DataExtractor": {
                    "tokens_input": 2340, "tokens_output": 180, "calls": 3,
                    "success_rate": 67, "avg_response_time": 15.2, "retries": 2
                },
                "Программист": {
                    "tokens_input": 3120, "tokens_output": 1240, "calls": 2,
                    "success_rate": 50, "avg_response_time": 12.8, "retries": 1
                },
                "Ревьюер": {
                    "tokens_input": 2890, "tokens_output": 1420, "calls": 3,
                    "success_rate": 67, "avg_response_time": 18.3, "retries": 2
                },
                "TeamLead": {
                    "tokens_input": 1650, "tokens_output": 420, "calls": 1,
                    "success_rate": 100, "avg_response_time": 11.2, "retries": 0
                },
                "ТехПисатель": {
                    "tokens_input": 1340, "tokens_output": 640, "calls": 1,
                    "success_rate": 100, "avg_response_time": 9.7, "retries": 0
                }
            },
            "improvement_cycles": 3,
            "teamlead_interventions": 1,
            "final_test_success": True
        }
        
        self.token_cost_input = 0.0002   # руб/токен
        self.token_cost_output = 0.0002  # руб/токен
    
    def calculate_costs(self):
        """Расчет экономических метрик"""
        costs_by_agent = {}
        total_cost = 0
        
        for agent, data in self.session_data["agents_performance"].items():
            input_cost = data["tokens_input"] * self.token_cost_input
            output_cost = data["tokens_output"] * self.token_cost_output
            agent_cost = input_cost + output_cost
            
            costs_by_agent[agent] = {
                "total_tokens": data["tokens_input"] + data["tokens_output"],
                "cost_rubles": round(agent_cost, 4),
                "calls": data["calls"],
                "cost_per_call": round(agent_cost / data["calls"], 4)
            }
            total_cost += agent_cost
        
        return costs_by_agent, round(total_cost, 4)
    
    def analyze_efficiency(self):
        """Анализ эффективности агентов"""
        efficiency_metrics = {}
        
        for agent, data in self.session_data["agents_performance"].items():
            efficiency_metrics[agent] = {
                "success_rate": data["success_rate"],
                "retries": data["retries"],
                "efficiency_score": data["success_rate"] / (1 + data["retries"]),  # Простая формула
                "response_time": data["avg_response_time"]
            }
        
        return efficiency_metrics
    
    def generate_report(self):
        """Генерация полного отчета"""
        costs, total_cost = self.calculate_costs()
        efficiency = self.analyze_efficiency()
        
        duration = self.session_data["end_time"] - self.session_data["start_time"]
        
        return {
            "summary": {
                "total_duration_minutes": round(duration.total_seconds() / 60, 1),
                "total_cost_rubles": total_cost,
                "improvement_cycles": self.session_data["improvement_cycles"],
                "teamlead_interventions": self.session_data["teamlead_interventions"],
                "success": self.session_data["final_test_success"]
            },
            "costs": costs,
            "efficiency": efficiency
        }

# Демонстрация анализа метрик
analyzer = MetricsAnalyzer()
report = analyzer.generate_report()

print("📊 ОТЧЕТ ПО ПРОИЗВОДИТЕЛЬНОСТИ МУЛЬТИАГЕНТНОЙ СИСТЕМЫ")
print("=" * 80)

# Общая сводка
summary = report["summary"]
print(f"⏱️  Общее время выполнения: {summary['total_duration_minutes']} минут")
print(f"💰 Общая стоимость: {summary['total_cost_rubles']} руб.")
print(f"🔄 Циклов улучшения: {summary['improvement_cycles']}")
print(f"👨‍💼 Вмешательств TeamLead: {summary['teamlead_interventions']}")
print(f"✅ Результат: {'УСПЕХ' if summary['success'] else 'ПРОВАЛ'}")

# Анализ по агентам
print(f"\n💳 СТОИМОСТЬ ПО АГЕНТАМ:")
costs_sorted = sorted(report["costs"].items(), key=lambda x: x[1]["cost_rubles"], reverse=True)
for agent, cost_data in costs_sorted:
    print(f"   {agent:.<15} {cost_data['cost_rubles']:>8.4f} руб ({cost_data['total_tokens']:>5} токенов, {cost_data['calls']} вызовов)")

print(f"\n⚡ ЭФФЕКТИВНОСТЬ АГЕНТОВ:")
efficiency_sorted = sorted(report["efficiency"].items(), key=lambda x: x[1]["efficiency_score"], reverse=True)
for agent, eff_data in efficiency_sorted:
    score = eff_data["efficiency_score"]
    retries = eff_data["retries"]
    success = eff_data["success_rate"]
    time_taken = eff_data["response_time"]
    
    print(f"   {agent:.<15} Эффективность: {score:>5.1f} | Успех: {success:>3}% | Повторы: {retries} | Время: {time_taken:>4.1f}с")

# Выводы и рекомендации
print(f"\n🎯 ВЫВОДЫ:")
most_expensive = costs_sorted[0]
least_efficient = efficiency_sorted[-1]

print(f"   💸 Самый затратный: {most_expensive[0]} ({most_expensive[1]['cost_rubles']} руб)")
print(f"   🐌 Наименее эффективный: {least_efficient[0]} (эффективность: {least_efficient[1]['efficiency_score']:.1f})")
print(f"   🎊 TeamLead вмешательства: {'Эффективны' if summary['teamlead_interventions'] <= 2 else 'Избыточны'}")

# ROI анализ
roi_score = 10 / summary['total_cost_rubles'] if summary['success'] else 0  # 10 руб ценности за успешный скрипт
print(f"   📈 ROI: {roi_score:.1f} (ценность/стоимость)")

print(f"\n🚀 РЕКОМЕНДАЦИИ:")
if least_efficient[1]["efficiency_score"] < 50:
    print(f"   • Улучшить промпты для {least_efficient[0]} (слишком много повторов)")
if summary['teamlead_interventions'] > 2:
    print(f"   • Настроить лучшую координацию между Программистом и Ревьюером")
if summary['total_cost_rubles'] > 1.0:
    print(f"   • Оптимизировать токены - проект дорогой для простой задачи")
print(f"   • Система работает эффективно, готова к продакшену!")

📊 ОТЧЕТ ПО ПРОИЗВОДИТЕЛЬНОСТИ МУЛЬТИАГЕНТНОЙ СИСТЕМЫ
⏱️  Общее время выполнения: 12.0 минут
💰 Общая стоимость: 3.408 руб.
🔄 Циклов улучшения: 3
👨‍💼 Вмешательств TeamLead: 1
✅ Результат: УСПЕХ

💳 СТОИМОСТЬ ПО АГЕНТАМ:
   Программист....   0.8720 руб ( 4360 токенов, 2 вызовов)
   Ревьюер........   0.8620 руб ( 4310 токенов, 3 вызовов)
   DataExtractor..   0.5040 руб ( 2520 токенов, 3 вызовов)
   TeamLead.......   0.4140 руб ( 2070 токенов, 1 вызовов)
   ТехПисатель....   0.3960 руб ( 1980 токенов, 1 вызовов)
   Архитектор.....   0.3600 руб ( 1800 токенов, 1 вызовов)

⚡ ЭФФЕКТИВНОСТЬ АГЕНТОВ:
   Архитектор..... Эффективность: 100.0 | Успех: 100% | Повторы: 0 | Время:  8.5с
   TeamLead....... Эффективность: 100.0 | Успех: 100% | Повторы: 0 | Время: 11.2с
   ТехПисатель.... Эффективность: 100.0 | Успех: 100% | Повторы: 0 | Время:  9.7с
   Программист.... Эффективность:  25.0 | Успех:  50% | Повторы: 1 | Время: 12.8с
   DataExtractor.. Эффективность:  22.3 | Успех:  67% | Повторы: 2 | Время:

## 🚀 Потенциал развития и направления улучшений

### Текущие ограничения системы:

#### 🔧 Технические ограничения:
- **Зависимость от качества промптов** - требует постоянной настройки
- **Ограниченная адаптивность** - фиксированные роли агентов
- **Простая логика координации** - нет сложных паттернов взаимодействия
- **Отсутствие обучения** - агенты не улучшаются от опыта

#### 📊 Ограничения мониторинга:
- **Базовые метрики** - нет глубокой аналитики поведения
- **Отсутствие предсказательной аналитики** - не предвидит проблемы
- **Ограниченный A/B тестинг** промптов и стратегий

### Направления развития:

#### 🧠 Улучшение ИИ компонентов:
- **Адаптивные промпты** - автоматическая настройка под задачи  
- **Обучение с подкреплением** - улучшение агентов на основе обратной связи  
- **Мета-агенты** - агенты, которые создают других агентов  
- **Персональные ассистенты** - запоминание предпочтений пользователя

#### 🔗 Расширение возможностей:
- **Поддержка других языков** программирования (Go, Rust, Java)  
- **Интеграция с Git** - автоматические коммиты и PR  
- **Работа с базами данных** - агенты для SQL и NoSQL  
- **DevOps агенты** - развертывание и мониторинг  
- **Security агенты** - анализ уязвимостей кода

#### 📈 Продвинутая аналитика:
- **Предиктивные модели** - прогнозирование сложности задач  
- **Оптимизация стоимости** - умное распределение ресурсов  
- **Качественные метрики** - оценка читаемости и maintainability кода  
- **Benchmark системы** - сравнение с человеческими разработчиками

### Ключевые инсайты о мультиагентных системах для разработки:

#### ✅ **Что работает хорошо:**
- **Специализация агентов** значительно повышает качество результатов
- **Иерархическое управление** с TeamLead эффективно решает конфликты
- **Строгая валидация данных** через Pydantic предотвращает ошибки
- **Итеративные циклы улучшения** приводят к стабильным решениям
- **Многоуровневое логирование** обеспечивает полную прозрачность процесса

#### ⚠️ **Основные сложности:**
- **Координация агентов** требует сложной логики и множества edge cases
- **Обработка ошибок** должна быть продумана для каждого типа сбоя
- **Стоимость токенов** может быстро расти при неэффективных промптах
- **Предотвращение циклов** требует интеллектуального анализа паттернов
- **Балансировка автономности** агентов и централизованного контроля

#### 🎯 **Практические рекомендации:**

1. **Начинайте с простых задач** - сложность растет экспоненциально
2. **Инвестируйте в качественные промпты** - это основа надежности
3. **Внедряйте мониторинг с первого дня** - без этого невозможна отладка
4. **Готовьтесь к итерационной разработке** - первая версия никогда не работает идеально
5. **Планируйте бюджет на токены** - production системы могут быть дорогими

### 🚀 **Будущее мультиагентных систем в разработке ПО:**

Мультиагентные системы представляют **парадигмальный сдвиг** в автоматизации разработки ПО. 
Вместо попыток создать одного "супер-разработчика-ИИ", мы строим **команды специализированных агентов**, 
которые взаимодействуют друг с другом подобно реальным командам разработчиков.

Это направление имеет огромный потенциал, но требует серьезных инвестиций в:
- Исследования координации агентов
- Стандартизацию протоколов взаимодействия  
- Создание надежных фреймворков
- Обучение специалистов новым подходам

In [24]:
print('well done')

well done
